In [1]:
%pip install python-dotenv pandas chardet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import_file = "George-CSV-Export/2025-01-01_2025-12-31_03.csv"

In [3]:
import chardet

# Open the CSV file in binary mode
with open(import_file, 'rb') as file:
    raw_data = file.read()

# Detect the encoding
result = chardet.detect(raw_data)

csv_encoding = result['encoding']

# Print the detected encoding
print(f"Detected encoding: {csv_encoding}")

Detected encoding: UTF-16


In [4]:
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv(import_file, encoding=csv_encoding, parse_dates=[
    "Buchungsdatum", 
    "Valutadatum"
    ], dayfirst=True, keep_default_na=False)


# apply data type fixes
df['Highlight'] = df['Highlight'].astype(bool)

# input number format: "10.123,02"
# output: 100123.02
df['Betrag'] = df['Betrag'].apply( lambda x: float(x.replace(".", "").replace(",", ".")))

# Display the first few rows of the DataFrame
df.head()

,Eigener Kontoname,Eigene IBAN,Buchungsdatum,Partnername,Partner IBAN,BIC/SWIFT,Partner Kontonummer,Bankleitzahl,Betrag,Währung,...,Buchungsreferenz,Notiz,Highlight,Valutadatum,Virtuelle Kartennummer,Bezahlt mit,App,Zahlungsreferenz,Mandats ID,Creditor ID
0,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2025-05-09,Sparkasse Salzkammergut,AT682031491718500118,SKBIAT21XXX,91718500118,20314,1000.00,EUR,...,203142505092AEI-7D493ADWQ2PE,,False,2025-05-09,,,,,,
1,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2025-05-07,Google Cloud EMEA Limited,IE35CITI99005132549640,CITIIE2XXXX,,,-34.50,EUR,...,201002505022AEI-43E7NO000902,,False,2025-05-07,,,,,51126908,IE34ZZZ362297
2,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2025-05-05,Erste Bank,AT052011140005191900,GIBAATWWXXX,40005191900,20111,-59.80,EUR,...,201112505022ALV-250502199032,,False,2025-05-05,,,,,231128ALV-202/05509090741,AT68ZZZ00000004435
3,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2025-04-07,Google Cloud EMEA Limited,IE35CITI99005132549640,CITIIE2XXXX,,,-34.50,EUR,...,201002504012AEI-18EYPF000626,,False,2025-04-07,,,,,51126908,IE34ZZZ362297
4,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2025-04-04,Erste Bank,AT052011140005191900,GIBAATWWXXX,40005191900,20111,-72.18,EUR,...,201112504012ALV-250401199072,,False,2025-04-04,,,,,231128ALV-202/05509090741,AT68ZZZ00000004435


In [5]:
df.dtypes

Eigener Kontoname                 object
Eigene IBAN                       object
Buchungsdatum             datetime64[ns]
Partnername                       object
Partner IBAN                      object
BIC/SWIFT                         object
Partner Kontonummer               object
Bankleitzahl                      object
Betrag                           float64
Währung                           object
Buchungs-Details                  object
Buchungsreferenz                  object
Notiz                             object
Highlight                           bool
Valutadatum               datetime64[ns]
Virtuelle Kartennummer            object
Bezahlt mit                       object
App                               object
Zahlungsreferenz                  object
Mandats ID                        object
Creditor ID                       object
dtype: object

In [6]:


import psycopg
import os
import pytz

os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        
        sql = """
        INSERT INTO accounting_booking (
            account_name,
            iban,
            booking_date,
            partner_name,
            partner_iban,
            bic_swift,
            partner_account_number,
            bank_code,
            amount,
            currency,
            booking_details,
            booking_reference,  -- unique reference
            note,
            highlight,
            value_date,
            virtual_card_number,
            paid_with,
            app,
            payment_reference,
            mandate_id,
            creditor_id
        )
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (booking_date, partner_iban, amount, currency, booking_details, value_date, mandate_id, creditor_id, payment_reference, booking_reference)
        DO NOTHING
        """


        for index, row in df.iterrows():
            values = (
                row['Eigener Kontoname'],  # account_name
                row['Eigene IBAN'],  # iban
                row['Buchungsdatum'],  # booking_date
                row['Partnername'],  # partner_name
                row['Partner IBAN'],  # partner_iban
                row['BIC/SWIFT'],  # bic_swift
                row['Partner Kontonummer'],  # partner_account_number
                row['Bankleitzahl'],  # bank_code
                row['Betrag'],  # amount
                row['Währung'],  # currency
                row['Buchungs-Details'],  # booking_details
                row['Buchungsreferenz'],  # booking_reference (unique key)
                row['Notiz'],  # note
                row['Highlight'],  # highlight
                row['Valutadatum'],  # value_date
                row['Virtuelle Kartennummer'],  # virtual_card_number
                row['Bezahlt mit'],  # paid_with
                row['App'],  # app
                row['Zahlungsreferenz'],  # payment_reference
                row['Mandats ID'],  # mandate_id
                row['Creditor ID']  # creditor_id
            )
            # Execute the SQL query for each row
            cur.execute(sql, values)
        
        conn.commit()


